## Diet Optimization

Welcome to the diet problem! The diet problem is one of the first large-scale optimization
problems to be studied in practice. Back in the 1930’s and 40’s, the Army wanted to meet the nutritional requirements of its soldiers while minimizing the cost. 


#### Part 1
1.	Formulate an optimization model (a linear program) to find the cheapest diet that satisfies the maximum and minimum daily nutrition constraints, and solve it using PuLP.  Turn in your code and the solution. (The optimal solution should be a diet of air-popped popcorn, poached eggs, oranges, raw iceberg lettuce, raw celery, and frozen broccoli. UGH!)

In [3]:
# ! pip install pulp
# ! pip install xlrd

from pulp import *
import pandas as pd

**Step 1:** Read and view the dataset

In [7]:
import pandas as pd

def read_excel_file(file_path):
    try:
        # Use pandas to read the Excel file
        data = pd.read_excel(file_path)

        # Process or display the data as needed
        print(data.head())

    except FileNotFoundError:
        print(f"File not found: {file_path}")

# Example usage:
# Replace 'your_excel_file.xlsx' with the actual file name
# Provide the full path if the file is not in the same directory as your script
file_path = '/Users/jemyap/Desktop/Projects-Data-Science/Optimization/data 15.2/diet.xls'


In [8]:
diet = pd.read_excel(file_path, sheet_name='Sheet1')
# diet.loc[diet['Foods'] == "Celery, Raw"] = "Celery,Raw"
diet

,Foods,Price/ Serving,Serving Size,Calories,Cholesterol mg,Total_Fat g,Sodium mg,Carbohydrates g,Dietary_Fiber g,Protein g,Vit_A IU,Vit_C IU,Calcium mg,Iron mg
0,Frozen Broccoli,0.16,10 Oz Pkg,73.8,0.0,0.8,68.2,13.6,8.5,8.0,5867.4,160.2,159.0,2.3
1,"Carrots,Raw",0.07,1/2 Cup Shredded,23.7,0.0,0.1,19.2,5.6,1.6,0.6,15471.0,5.1,14.9,0.3
2,"Celery,Raw",0.04,1 Stalk,6.4,0.0,0.1,34.8,1.5,0.7,0.3,53.6,2.8,16.0,0.2
3,Frozen Corn,0.18,1/2 Cup,72.2,0.0,0.6,2.5,17.1,2.0,2.5,106.6,5.2,3.3,0.3
4,"Lettuce,Iceberg,Raw",0.02,1 Leaf,2.6,0.0,0.0,1.8,0.4,0.3,0.2,66.0,0.8,3.8,0.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62,"Crm Mshrm Soup,W/Mlk",0.65,1 C (8 Fl Oz),203.4,19.8,13.6,1076.3,15.0,0.5,6.1,153.8,2.2,178.6,0.6
63,"Beanbacn Soup,W/Watr",0.67,1 C (8 Fl Oz),172.0,2.5,5.9,951.3,22.8,8.6,7.9,888.0,1.5,81.0,2.0
64,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
65,NaN,NaN,Minimum daily intake,1500.0,30.0,20.0,800.0,130.0,125.0,60.0,1000.0,400.0,700.0,10.0


**Step 2:** Extract the relevant dataset, and remove the maximum and minimum nutritional constraints. These constraints will be revisited when using the Pulp package.

In [3]:
# Remove max and min nutritional intake rows
diet_data = diet[0:64]
diet_data

,Foods,Price/ Serving,Serving Size,Calories,Cholesterol mg,Total_Fat g,Sodium mg,Carbohydrates g,Dietary_Fiber g,Protein g,Vit_A IU,Vit_C IU,Calcium mg,Iron mg
0,Frozen Broccoli,0.16,10 Oz Pkg,73.8,0.0,0.8,68.2,13.6,8.5,8.0,5867.4,160.2,159.0,2.3
1,"Carrots,Raw",0.07,1/2 Cup Shredded,23.7,0.0,0.1,19.2,5.6,1.6,0.6,15471.0,5.1,14.9,0.3
2,"Celery,Raw",0.04,1 Stalk,6.4,0.0,0.1,34.8,1.5,0.7,0.3,53.6,2.8,16.0,0.2
3,Frozen Corn,0.18,1/2 Cup,72.2,0.0,0.6,2.5,17.1,2.0,2.5,106.6,5.2,3.3,0.3
4,"Lettuce,Iceberg,Raw",0.02,1 Leaf,2.6,0.0,0.0,1.8,0.4,0.3,0.2,66.0,0.8,3.8,0.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,Neweng Clamchwd,0.75,1 C (8 Fl Oz),175.7,10.0,5.0,1864.9,21.8,1.5,10.9,20.1,4.8,82.8,2.8
60,Tomato Soup,0.39,1 C (8 Fl Oz),170.7,0.0,3.8,1744.4,33.2,1.0,4.1,1393.0,133.0,27.6,3.5
61,"New E Clamchwd,W/Mlk",0.99,1 C (8 Fl Oz),163.7,22.3,6.6,992.0,16.6,1.5,9.5,163.7,3.5,186.0,1.5
62,"Crm Mshrm Soup,W/Mlk",0.65,1 C (8 Fl Oz),203.4,19.8,13.6,1076.3,15.0,0.5,6.1,153.8,2.2,178.6,0.6


**Step 3:** Construct relevant data structures to build the constraints required for this optimisation problem. For example, each nutritional intake will have its own dictionary of nutritional informatino for each food in the list.

In [4]:
data = {
    'Foods' : diet_data['Foods'],
    'Price/ Serving' : diet_data['Price/ Serving'],
    'Calories' : diet_data['Calories'],
    'Cholesterol mg' : diet_data['Cholesterol mg'],
    'Total_Fat g' : diet_data['Total_Fat g'],
    'Sodium mg' : diet_data['Sodium mg'],
    'Carbohydrates g' : diet_data['Carbohydrates g'], 
    'Dietary_Fiber g' : diet_data['Dietary_Fiber g'],
    'Protein g' : diet_data['Protein g'],
    'Vit_A IU' : diet_data['Vit_A IU'],
    'Vit_C IU' : diet_data['Vit_C IU'],
    'Calcium mg' : diet_data['Calcium mg'],
    'Iron mg' : diet_data['Iron mg']
}

# Create list of foods
foods = diet_data['Foods'].tolist() 

# Create dictionary for cost of each foods item
costs = dict(zip(data['Foods'], data['Price/ Serving']))

# Create dictionaries for each nutritional requirement
calories_dict = dict(zip(data['Foods'], data['Calories']))
cholesterol_dict = dict(zip(data['Foods'], data['Cholesterol mg']))
fat_dict = dict(zip(data['Foods'], data['Total_Fat g']))
sodium_dict = dict(zip(data['Foods'], data['Sodium mg']))
carbohydrates_dict = dict(zip(data['Foods'], data['Carbohydrates g']))
fiber_dict = dict(zip(data['Foods'], data['Dietary_Fiber g']))
protein_dict = dict(zip(data['Foods'], data['Protein g']))
vit_a_dict = dict(zip(data['Foods'], data['Vit_A IU']))
vit_c_dict = dict(zip(data['Foods'], data['Vit_C IU']))
calcium_dict = dict(zip(data['Foods'], data['Calcium mg']))
iron_dict = dict(zip(data['Foods'], data['Iron mg']))


**Step 4:** Define the objective function, and the goal to minimise it, as well as include the food variables which the model will find values to. These numbers will be the solution for this optimisation problem.

In [5]:
# Create 'prob' variable to contain problem data - minimisation problem
prob = LpProblem("Diet Problem", LpMinimize)

# Create food variables
food_vars = LpVariable.dicts("food", foods, 0)

# Add objective function to 'prob' first
prob += (
    lpSum([costs[i] * food_vars[i] for i in foods]),
    "Total cost of diet"
)



/Users/jemyap/opt/anaconda3/lib/python3.9/site-packages/pulp/pulp.py:1352: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


**Step 5:** Here, the constraints for each nutritional item are included in the model. Both min and max nutritional intake, and for all nutritional items.

In [6]:
# Define other constraints, namely maximum and minimum nutritional intake

min_intake = [1500, 30, 20, 800, 130, 125, 60, 1000, 400, 700, 10]

# Add minimum nutritional intake constraints
prob += (
    lpSum([calories_dict[i] * food_vars[i] for i in foods]) >= 1500,
    "CalorieIntake",
)

prob += (
    lpSum([cholesterol_dict[i] * food_vars[i] for i in foods]) >= 30,
    "Cholesterol",
)

prob += (
    lpSum([fat_dict[i] * food_vars[i] for i in foods]) >= 20,
    "Fat",
)

prob += (
    lpSum([sodium_dict[i] * food_vars[i] for i in foods]) >= 800,
    "Sodium",
)

prob += (
    lpSum([carbohydrates_dict[i] * food_vars[i] for i in foods]) >= 130,
    "Carbohydrates",
)

prob += (
    lpSum([fiber_dict[i] * food_vars[i] for i in foods]) >= 125,
    "Fibre",
)

prob += (
    lpSum([protein_dict[i] * food_vars[i] for i in foods]) >= 60,
    "Protein",
)

prob += (
    lpSum([vit_a_dict[i] * food_vars[i] for i in foods]) >= 1000,
    "VitaminA",
)

prob += (
    lpSum([vit_c_dict[i] * food_vars[i] for i in foods]) >= 400,
    "VitaminC",
)

prob += (
    lpSum([calcium_dict[i] * food_vars[i] for i in foods]) >= 700,
    "Calcium",
)

prob += (
    lpSum([iron_dict[i] * food_vars[i] for i in foods]) >= 10,
    "Iron",
)



In [7]:
# Add maximum nutritional intake constraints

max_intake = [2400, 240, 70, 2000, 450, 250, 100, 10000, 5000, 1500, 40]

prob += (
    lpSum([calories_dict[i] * food_vars[i] for i in foods]) <= 2400,
    "MaxCalorieIntake",
)

prob += (
    lpSum([cholesterol_dict[i] * food_vars[i] for i in foods]) <= 240,
    "MaxCholesterol",
)

prob += (
    lpSum([fat_dict[i] * food_vars[i] for i in foods]) <= 70,
    "MaxFat",
)

prob += (
    lpSum([sodium_dict[i] * food_vars[i] for i in foods]) <= 2000,
    "MaxSodium",
)

prob += (
    lpSum([carbohydrates_dict[i] * food_vars[i] for i in foods]) <= 450,
    "MaxCarbohydrates",
)

prob += (
    lpSum([fiber_dict[i] * food_vars[i] for i in foods]) <= 250,
    "MaxFibre",
)

prob += (
    lpSum([protein_dict[i] * food_vars[i] for i in foods]) <= 100,
    "MaxProtein",
)

prob += (
    lpSum([vit_a_dict[i] * food_vars[i] for i in foods]) <= 10000,
    "MaxVitaminA",
)

prob += (
    lpSum([vit_c_dict[i] * food_vars[i] for i in foods]) <= 5000,
    "MaxVitaminC",
)

prob += (
    lpSum([calcium_dict[i] * food_vars[i] for i in foods]) <= 1500,
    "MaxCalcium",
)

prob += (
    lpSum([iron_dict[i] * food_vars[i] for i in foods]) <= 40,
    "MaxIron",
)

In [8]:
# Problem data is written to a .lp file

prob.writeLP("DietModel.lp")

[food_2%_Lowfat_Milk,
 food_3.3%_Fat,Whole_Milk,
 food_Apple,Raw,W_Skin,
 food_Apple_Pie,
 food_Bagels,
 food_Banana,
 food_Beanbacn_Soup,W_Watr,
 food_Bologna,Turkey,
 food_Butter,Regular,
 food_Cap'N_Crunch,
 food_Carrots,Raw,
 food_Celery,Raw,
 food_Cheddar_Cheese,
 food_Cheerios,
 food_Chicknoodl_Soup,
 food_Chocolate_Chip_Cookies,
 food_Corn_Flks,_Kellogg'S,
 food_Couscous,
 food_Crm_Mshrm_Soup,W_Mlk,
 food_Frankfurter,_Beef,
 food_Frozen_Broccoli,
 food_Frozen_Corn,
 food_Grapes,
 food_Ham,Sliced,Extralean,
 food_Hamburger_W_Toppings,
 food_Hotdog,_Plain,
 food_Kielbasa,Prk,
 food_Kiwifruit,Raw,Fresh,
 food_Lettuce,Iceberg,Raw,
 food_Macaroni,Ckd,
 food_Malt_O_Meal,Choc,
 food_New_E_Clamchwd,W_Mlk,
 food_Neweng_Clamchwd,
 food_Oatmeal,
 food_Oatmeal_Cookies,
 food_Oranges,
 food_Peanut_Butter,
 food_Peppers,_Sweet,_Raw,
 food_Pizza_W_Pepperoni,
 food_Poached_Eggs,
 food_Popcorn,Air_Popped,
 food_Pork,
 food_Potato_Chips,Bbqflvr,
 food_Potatoes,_Baked,
 food_Pretzels,
 food_Raisin

**Step 6:** Finally, we solve the problem and print out the solution using a 'for loop'.

In summary, the recipe contains:

<br> 1. 52.64 servings of Celery </br>
<br> 2. 0.26 servings of Broccoli </br>
<br> 3. 64.00 servings of Iceberg Lettuce </br>
<br> 4. 2.29 servings of Oranges </br>
<br> 5. 0.14 servings of poached eggs </br>
<br> 6. 13.87 servings of popcorn </br>

At a cost of $4.34 per person

In [9]:
prob.solve()
print("Status:", LpStatus[prob.status])


Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/jemyap/opt/anaconda3/lib/python3.9/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/pg/rfrygnnj6js6hvd_b_90mn5w0000gn/T/5a5ec37167f24ca38851e9d8968da963-pulp.mps timeMode elapsed branch printingOptions all solution /var/folders/pg/rfrygnnj6js6hvd_b_90mn5w0000gn/T/5a5ec37167f24ca38851e9d8968da963-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 27 COLUMNS
At line 1286 RHS
At line 1309 BOUNDS
At line 1310 ENDATA
Problem MODEL has 22 rows, 64 columns and 1194 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 22 (0) rows, 64 (0) columns and 1194 (0) elements
0  Obj 0 Primal inf 21.63092 (11)
9  Obj 4.3371168
Optimal - objective value 4.3371168
Optimal objective 4.33711681 - 9 iterations time 0.002
Option for printingOptions changed from normal to all
Total time (CPU seconds):       0.00   (Wallclock sec

In [10]:
for v in prob.variables():
    if v.varValue != 0.0:
        print(v.name, "=", v.varValue)

print ("Total cost of diet : $", round(value(prob.objective),2), sep='')


food_Celery,Raw = 52.64371
food_Frozen_Broccoli = 0.25960653
food_Lettuce,Iceberg,Raw = 63.988506
food_Oranges = 2.2929389
food_Poached_Eggs = 0.14184397
food_Popcorn,Air_Popped = 13.869322
Total cost of diet : $4.34


#### Part 2

I will add to my model the following constraints (which might require adding more variables) and solve the new model:

a.	If a food is selected, then a minimum of 1/10 serving must be chosen.

b.	Many people dislike celery and frozen broccoli. So at most one, but not both, can be selected.

c.	To get day-to-day variety in protein, at least 3 kinds of meat/poultry/fish/eggs must be selected. [If something is ambiguous (e.g., should bean-and-bacon soup be considered meat?).

In [11]:
chosenvar = LpVariable.dicts("chosen", foods, 0, 1, "Binary")

### Key considerations for introduction of additional constraints
For part 2 of the assignment, we add in 3 more constraints. The first constraint pertaining to minimum serving size requires a pair of constraints with a minimum and maximum serving. Without the maximum serving constraint, the program will set the binary variable "chosenvar" to 0, therefore avoiding the minimum serving constraint. With a maximum constraint introduced, the program will be unable to use this solution to solve for the optimisation problem.

We then add in the other two constraints.

In [12]:
# Add constraint a

for food in foods:
    prob += (
        food_vars[food] >= 0.1 * chosenvar[food],
        f"MinServing_{food}"
    )

    prob += (
        food_vars[food] <= 10000 * chosenvar[food],
        f"MaxServing_{food}"
    )

# Add constraint b
prob += (
    (chosenvar['Frozen Broccoli'] + chosenvar['Celery,Raw']) <= 1,
    'Either Broccoli or Celery'
)

# Add constraint c

prob += (
    chosenvar['Roasted Chicken'] + chosenvar['Poached Eggs'] + \
    chosenvar['Scrambled Eggs'] + chosenvar['Frankfurter, Beef'] + \
    chosenvar['Kielbasa,Prk'] + chosenvar['Hamburger W/Toppings'] + \
    chosenvar['Hotdog, Plain'] + chosenvar['Pork'] + \
    chosenvar['Bologna,Turkey'] + chosenvar['Ham,Sliced,Extralean'] + \
    chosenvar['White Tuna in Water'] \
    >= 3, 'At least three proteins'
)




We solve the problem and output the solution. With these additional constraints, we get a diet that:
1) has food serving sizes of at least 0.1
2) has more protein (at least 3 sources);
3) has either broccoli or celery (but not both);

As a result, the cost of this diet that contains more protein sources has increased to $4.51

In [13]:
prob.solve()
print("Status:", LpStatus[prob.status])


Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/jemyap/opt/anaconda3/lib/python3.9/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/pg/rfrygnnj6js6hvd_b_90mn5w0000gn/T/eb23dd08bab74580a839f17554331c0a-pulp.mps timeMode elapsed branch printingOptions all solution /var/folders/pg/rfrygnnj6js6hvd_b_90mn5w0000gn/T/eb23dd08bab74580a839f17554331c0a-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 157 COLUMNS
At line 1813 RHS
At line 1966 BOUNDS
At line 2031 ENDATA
Problem MODEL has 152 rows, 128 columns and 1463 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 4.38006 - 0.00 seconds
Cgl0003I 0 fixed, 0 tightened bounds, 64 strengthened rows, 0 substitutions
Cgl0004I processed model has 141 rows, 128 columns (64 integer (64 of which binary)) and 866 elements
Cbc0038I Initial state - 8 integers unsatisfied sum - 1.63213
Cbc0038I Pas

In [14]:
print('Optimized Diet:', '\n')
for v in prob.variables():
    if v.varValue > 0:
       if v.name.startswith('food_'):
            print(v.name, "=", v.varValue)

print ("Total cost of diet : $", round(value(prob.objective),2), sep='')

Optimized Diet: 

food_Celery,Raw = 45.344194
food_Kielbasa,Prk = 0.1
food_Lettuce,Iceberg,Raw = 79.551746
food_Oranges = 2.996196
food_Peanut_Butter = 1.3207888
food_Poached_Eggs = 0.1
food_Popcorn,Air_Popped = 13.332948
food_Scrambled_Eggs = 0.1
Total cost of diet : $4.51


## A Larger Problem..

Now I will try to optimize for a larger data set - file diet_large.xls, which is a low-cholesterol diet model (rather than minimizing cost, the goal is to minimize cholesterol intake). 


We take a look at the data and drop the last few junk rows.

In [9]:
file_path = "/Users/jemyap/Desktop/Projects-Data-Science/Optimization/data 15.2/diet_large.xls"
large_diet = pd.read_excel(file_path, header=1)
large_diet = large_diet.fillna(0)
large_diet = large_diet.drop_duplicates(subset='Long_Desc', keep='first')
large_diet.head()

,Long_Desc,Protein,"Carbohydrate, by difference",Energy,Water,Energy.1,"Calcium, Ca","Iron, Fe","Magnesium, Mg","Phosphorus, P",...,Riboflavin,Niacin,Pantothenic acid,Vitamin B-6,"Folate, total",Vitamin B-12,Vitamin K (phylloquinone),Cholesterol,"Fatty acids, total trans","Fatty acids, total saturated"
0,"Butter, salted",0.85,0.06,717,15.87,3000.0,24,0.02,2,24,...,0.034,0.042,0.11,0.003,3,0.17,7,215.0,0.0,51.368
1,"Butter, whipped, with salt",0.85,0.06,717,15.87,2999.0,24,0.16,2,23,...,0.034,0.042,0.11,0.003,3,0.13,7,219.0,0.0,50.489
2,"Butter oil, anhydrous",0.28,0,876,0.24,3665.0,4,0,0,3,...,0.005,0.003,0.01,0.001,0,0.01,8.6,256.0,0.0,61.924
3,"Cheese, blue",21.4,2.34,353,42.41,1477.0,528,0.31,23,387,...,0.382,1.016,1.729,0.166,36,1.22,2.4,75.0,0.0,18.669
4,"Cheese, brick",23.24,2.79,371,41.11,1552.0,674,0.43,24,451,...,0.351,0.118,0.288,0.065,20,1.26,2.5,94.0,0.0,18.764


In [16]:
large_diet = large_diet.iloc[:-4]
large_diet.tail(5)

,Long_Desc,Protein,"Carbohydrate, by difference",Energy,Water,Energy.1,"Calcium, Ca","Iron, Fe","Magnesium, Mg","Phosphorus, P",...,Riboflavin,Niacin,Pantothenic acid,Vitamin B-6,"Folate, total",Vitamin B-12,Vitamin K (phylloquinone),Cholesterol,"Fatty acids, total trans","Fatty acids, total saturated"
7138,Polydextrose,0,96,100,4,418.0,0,0,0,0,...,0,0,0,0,0,0,0,0.0,0.0,0.000
7139,"Fruit-flavored drink mix, powder, unsweetened",0,91.3,226,0.96,944.0,1105,0.03,0,509,...,0,0.001,0,0,0,0,0,0.0,0.0,0.000
7140,Vital wheat gluten,75.16,13.79,370,8.2,1548.0,142,5.2,25,260,...,0,0,0,0,0,0,0,0.0,0.0,0.272
7141,"Frog legs, raw",16.4,0,73,81.9,305.0,18,1.5,20,147,...,0.25,1.2,0,0.12,15,0.4,0.1,50.0,0.0,0.076
7142,"Fish, mackerel, salted",18.5,0,305,43,1276.0,66,1.4,60,254,...,0.19,3.3,0,0.41,15,12,7.8,95.0,0.0,7.148


Next we construct the relevant data structures to set up our problem.

In [17]:
large_diet_list = large_diet.values.tolist()

In [18]:
# Make dictionaries from the large list using dict comprehensions

protein = {x[0]:x[1] for x in large_diet_list}
carbs = {x[0]:x[2] for x in large_diet_list}
energy = {x[0]:x[3] for x in large_diet_list}
water = {x[0]:x[4] for x in large_diet_list}
calcium = {x[0]:x[6] for x in large_diet_list}
iron = {x[0]:x[7] for x in large_diet_list}
magnesium = {x[0]:x[8] for x in large_diet_list}
phosphorus = {x[0]:x[9] for x in large_diet_list}
potassium = {x[0]:x[10] for x in large_diet_list}
sodium = {x[0]:x[11] for x in large_diet_list}
zinc = {x[0]:x[12] for x in large_diet_list}
copper = {x[0]:x[13] for x in large_diet_list}
manganese = {x[0]:x[14] for x in large_diet_list}
selenium = {x[0]:x[15] for x in large_diet_list}
vit_a = {x[0]:x[16] for x in large_diet_list}
vit_e = {x[0]:x[17] for x in large_diet_list}
vit_d = {x[0]:x[18] for x in large_diet_list}
vit_c = {x[0]:x[19] for x in large_diet_list}
thiamin = {x[0]:x[20] for x in large_diet_list}
riboflavin = {x[0]:x[21] for x in large_diet_list}
thiamin = {x[0]:x[20] for x in large_diet_list}
niacin = {x[0]:x[22] for x in large_diet_list}
pantothenica_acid = {x[0]:x[23] for x in large_diet_list}
vit_b6 = {x[0]:x[24] for x in large_diet_list}
folate = {x[0]:x[25] for x in large_diet_list}
vit_b12 = {x[0]:x[26] for x in large_diet_list}
vit_k = {x[0]:x[27] for x in large_diet_list}
fattyacid_trans = {x[0]:x[29] for x in large_diet_list}
fattyacid_trans = {x[0]:x[30] for x in large_diet_list}

cholesterol = {x[0]:x[28] for x in large_diet_list}

food_list = [x[0] for x in large_diet_list]


We then build the model, including the objective function.

In [19]:
# Create 'prob' variable to contain problem data - minimisation problem
probcholesterol = LpProblem("Cholesterol Diet Problem", LpMinimize)

# Create food variables
food_vars2 = LpVariable.dicts("food", food_list, 0)

# Add objective function to 'prob' first
probcholesterol += (
    lpSum([cholesterol[i] * food_vars2[i] for i in food_list]),
    "Total cholesterol in diet"
)


/Users/jemyap/opt/anaconda3/lib/python3.9/site-packages/pulp/pulp.py:1352: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


Next, we add in the constraints for minimum and maximum intake of each type of nutrition.

In [20]:
# Add constraints - min intake

probcholesterol += (
    lpSum([protein[i] * food_vars2[i] for i in food_list]) >= 56,
    "min protein"
)

probcholesterol += (
    lpSum([carbs[i] * food_vars2[i] for i in food_list]) >= 130,
    "min carbohydrates"
)

probcholesterol += (
    lpSum([energy[i] * food_vars2[i] for i in food_list]) >= 2400,
    "min energy"
)

probcholesterol += (
    lpSum([water[i] * food_vars2[i] for i in food_list]) >= 3700,
    "min water"
)

probcholesterol += (
    lpSum([calcium[i] * food_vars2[i] for i in food_list]) >= 1000,
    "min calcium"
)

probcholesterol += (
    lpSum([iron[i] * food_vars2[i] for i in food_list]) >= 8,
    "min iron"
)

probcholesterol += (
    lpSum([magnesium[i] * food_vars2[i] for i in food_list]) >= 270,
    "min magnesium"
)

probcholesterol += (
    lpSum([phosphorus[i] * food_vars2[i] for i in food_list]) >= 700,
    "min phosphorus"
)

probcholesterol += (
    lpSum([potassium[i] * food_vars2[i] for i in food_list]) >= 4700,
    "min potassium"
)

probcholesterol += (
    lpSum([sodium[i] * food_vars2[i] for i in food_list]) >= 1500,
    "min sodium"
)

probcholesterol += (
    lpSum([zinc[i] * food_vars2[i] for i in food_list]) >= 11,
    "min zinc"
)
probcholesterol += (
    lpSum([copper[i] * food_vars2[i] for i in food_list]) >= 0.9,
    "min copper"
)

probcholesterol += (
    lpSum([manganese[i] * food_vars2[i] for i in food_list]) >= 2.3,
    "min manganese"
)

probcholesterol += (
    lpSum([selenium[i] * food_vars2[i] for i in food_list]) >= 55,
    "min selenium"
)

probcholesterol += (
    lpSum([vit_a[i] * food_vars2[i] for i in food_list]) >= 900,
    "min vit a"
)

probcholesterol += (
    lpSum([vit_e[i] * food_vars2[i] for i in food_list]) >= 15,
    "min vit e"
)

probcholesterol += (
    lpSum([vit_d[i] * food_vars2[i] for i in food_list]) >= 200,
    "min vit d"
)

probcholesterol += (
    lpSum([vit_c[i] * food_vars2[i] for i in food_list]) >= 90,
    "min vit c"
)

probcholesterol += (
    lpSum([thiamin[i] * food_vars2[i] for i in food_list]) >= 0.0012,
    "min thiamin"
)

probcholesterol += (
    lpSum([riboflavin[i] * food_vars2[i] for i in food_list]) >= 1.3,
    "min riboflavin"
)

probcholesterol += (
    lpSum([niacin[i] * food_vars2[i] for i in food_list]) >= 16,
    "min niacin"
)

probcholesterol += (
    lpSum([pantothenica_acid[i] * food_vars2[i] for i in food_list]) >= 5,
    "min pantothenica_acid"
)

probcholesterol += (
    lpSum([vit_b6[i] * food_vars2[i] for i in food_list]) >= 1.3,
    "min vit_b6"
)

probcholesterol += (
    lpSum([folate[i] * food_vars2[i] for i in food_list]) >= 400,
    "min folate"
)

probcholesterol += (
    lpSum([vit_b12[i] * food_vars2[i] for i in food_list]) >= 2.4,
    "min vit b12"
)

probcholesterol += (
    lpSum([vit_k[i] * food_vars2[i] for i in food_list]) >= 120,
    "min vit k"
)

In [21]:
# Add constraints - max intake

probcholesterol += (
    lpSum([protein[i] * food_vars2[i] for i in food_list]) <= 1000000,
    "max protein"
)

probcholesterol += (
    lpSum([carbs[i] * food_vars2[i] for i in food_list]) <= 1000000,
    "max carbohydrates"
)

probcholesterol += (
    lpSum([energy[i] * food_vars2[i] for i in food_list]) <= 1000000,
    "max energy"
)

probcholesterol += (
    lpSum([water[i] * food_vars2[i] for i in food_list]) <= 1000000,
    "max water"
)

probcholesterol += (
    lpSum([calcium[i] * food_vars2[i] for i in food_list]) <= 2500,
    "max calcium"
)

probcholesterol += (
    lpSum([iron[i] * food_vars2[i] for i in food_list]) <= 45,
    "max iron"
)

probcholesterol += (
    lpSum([magnesium[i] * food_vars2[i] for i in food_list]) <= 400,
    "max magnesium"
)

probcholesterol += (
    lpSum([phosphorus[i] * food_vars2[i] for i in food_list]) <= 4000,
    "max phosphorus"
)

probcholesterol += (
    lpSum([potassium[i] * food_vars2[i] for i in food_list]) <= 1000000,
    "max potassium"
)

probcholesterol += (
    lpSum([sodium[i] * food_vars2[i] for i in food_list]) <= 2300,
    "max sodium"
)

probcholesterol += (
    lpSum([zinc[i] * food_vars2[i] for i in food_list]) <= 40,
    "max zinc"
)
probcholesterol += (
    lpSum([copper[i] * food_vars2[i] for i in food_list]) <= 10,
    "max copper"
)

probcholesterol += (
    lpSum([manganese[i] * food_vars2[i] for i in food_list]) <= 11,
    "max manganese"
)

probcholesterol += (
    lpSum([selenium[i] * food_vars2[i] for i in food_list]) <= 400,
    "max selenium"
)

probcholesterol += (
    lpSum([vit_a[i] * food_vars2[i] for i in food_list]) <= 3000,
    "max vit a"
)

probcholesterol += (
    lpSum([vit_e[i] * food_vars2[i] for i in food_list]) <= 1000,
    "max vit e"
)

probcholesterol += (
    lpSum([vit_d[i] * food_vars2[i] for i in food_list]) <= 2000,
    "max vit d"
)

probcholesterol += (
    lpSum([vit_c[i] * food_vars2[i] for i in food_list]) <= 2000,
    "max vit c"
)

probcholesterol += (
    lpSum([thiamin[i] * food_vars2[i] for i in food_list]) <= 1000000,
    "max thiamin"
)

probcholesterol += (
    lpSum([riboflavin[i] * food_vars2[i] for i in food_list]) <= 1000000,
    "max riboflavin"
)

probcholesterol += (
    lpSum([niacin[i] * food_vars2[i] for i in food_list]) <= 35,
    "max niacin"
)

probcholesterol += (
    lpSum([pantothenica_acid[i] * food_vars2[i] for i in food_list]) <= 1000000,
    "max pantothenica_acid"
)

probcholesterol += (
    lpSum([vit_b6[i] * food_vars2[i] for i in food_list]) <= 100,
    "max vit_b6"
)

probcholesterol += (
    lpSum([folate[i] * food_vars2[i] for i in food_list]) <= 1000,
    "max folate"
)

probcholesterol += (
    lpSum([vit_b12[i] * food_vars2[i] for i in food_list]) <= 1000000,
    "max vit b12"
)

probcholesterol += (
    lpSum([vit_k[i] * food_vars2[i] for i in food_list]) <= 1000000,
    "max vit k"
)

I also decided to include the minimum serving of food and set it to 0.1 as per the previous problem. 

In [22]:
chosenvar2 = LpVariable.dicts("chosen", food_list, 0, 1, "Binary")

In [23]:
for food in food_list:
    probcholesterol += (
        food_vars2[food] >= 0.1 * chosenvar2[food],
        f"MinServing_{food}"
    )

    probcholesterol += (
        food_vars2[food] <= 10000 * chosenvar2[food],
        f"MaxServing_{food}"
    )


Here, we get the model to find the optimal solution.

In [24]:
probcholesterol.solve()
print("Status:", LpStatus[probcholesterol.status])

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/jemyap/opt/anaconda3/lib/python3.9/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/pg/rfrygnnj6js6hvd_b_90mn5w0000gn/T/58f49a99135c4a02ac57991dbefa78da-pulp.mps timeMode elapsed branch printingOptions all solution /var/folders/pg/rfrygnnj6js6hvd_b_90mn5w0000gn/T/58f49a99135c4a02ac57991dbefa78da-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 12623 COLUMNS
At line 308206 RHS
At line 320825 BOUNDS
At line 327109 ENDATA
Problem MODEL has 12618 rows, 12566 columns and 279998 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 0 - 0.05 seconds
Cgl0004I processed model has 12576 rows, 12550 columns (6275 integer (6275 of which binary)) and 152407 elements
Cbc0038I Initial state - 7 integers unsatisfied sum - 1.06023
Cbc0038I Pass   1: (0.92 seconds) suminf.    0.10741 (5) obj. 9.83995

Interestingly, we get a solution where **infant formula** is introduced into the diet. Since the constraints did not restrict certain food types to be included in this low cholesterol diet, infant formula somehow got into the optimal diet for low cholesterol. And as per the objective function, a diet with 0 cholesterol was formulated.

In [25]:
print('Cholesterol-Optimized Diet:', '\n')
for v in probcholesterol.variables():
    if v.varValue > 0:
       if v.name.startswith('food_'):
            print(v.name, "=", v.varValue)

print ("Total cholesterol in diet : ", round(value(probcholesterol.objective),2), sep='')

Cholesterol-Optimized Diet: 

food_Infant_formula,_MEAD_JOHNSON,_ENFAMIL,_NUTRAMIGEN,_with_iron,_p = 0.92465402
food_Infant_formula,_MEAD_JOHNSON,_NEXT_STEP_PROSOBEE,_powder,_not_r = 2.12961
food_Leavening_agents,_yeast,_baker's,_compressed = 0.90457758
food_Salad_dressing,_home_recipe,_vinegar_and_oil = 1.279497
food_Tomatoes,_sun_dried = 0.80894962
food_Vegetable_oil,_avocado = 0.1
food_Water,_bottled,_non_carbonated,_CALISTOGA = 9998.5599
Total cholesterol in diet : 0.0


In [27]:
!jupyter nbconvert --to script HW11.ipynb

[NbConvertApp] Converting notebook HW11.ipynb to script
[NbConvertApp] Writing 21725 bytes to HW11.py
